MNIST 데이터를 활용해 컨볼루션 계층을 구현해 보겠습니다.
딥러닝이므로 신경망 
은닉층을 2개로 구성하겠습니다.

컨볼루션 계층을 구현하기 위해 다음 두 개 라이브러리 임포트가 필요합니다.

In [1]:
import tensorflow as tf
import numpy as np

텐서플로 1.7.0 버전에서부터는 샘플 데이터를 다운로드하는 기능이 제외될 예정이라는 경고가 발생합니다. 대신 케라스(Keras)를 사용하여 MNIST 데이터를 다운받습니다.

In [2]:
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()
X_train = X_train.astype(np.float32).reshape(-1, 28*28) / 255.0
X_test = X_test.astype(np.float32).reshape(-1, 28*28) / 255.0
y_train = tf.keras.utils.to_categorical(y_train)
y_test = tf.keras.utils.to_categorical(y_test)
y_train = y_train.astype(np.int32)
y_test = y_test.astype(np.int32)

배치 데이터를 만들기 위해 파이썬 제너레이터 함수를 정의합니다.

In [3]:
def shuffle_batch(X, y, batch_size):
    rnd_idx = np.random.permutation(len(X))
    n_batches = len(X) // batch_size
    for batch_idx in np.array_split(rnd_idx, n_batches):
        X_batch, y_batch = X[batch_idx], y[batch_idx]
        yield X_batch, y_batch

훈련 이미지 데이터를 넣을 플레이스홀더를 2차원 텐서로 만듭니다. 텐서 x는 이미지를 784픽셀을 실수로 저장하는 벡터로 사용됩니다. None이라고 지정한 것은 어떤 크기나 가능하다는 뜻으로서, 여기에서는 학습 과정에 사용될 이미지의 총 개수가 될 것입니다.
y_는 실제 레이블을 담는 프레이스홀더 입니다.
x 를 이미지 사이즈인 28x28x1 크기로 차원을 변경합니다.

In [4]:
x = tf.placeholder("float", shape=[None, 784])
y_ = tf.placeholder("float", shape=[None, 10])

x_image = tf.reshape(x, [-1,28,28,1])
print("x_image=", x_image)

x_image= Tensor("Reshape:0", shape=(?, 28, 28, 1), dtype=float32)


행렬 W와 편향 b와 정의하는 함수를 구현합니다. 
가중치는 표준편차를 0.1로 갖는 난수로 초기화하여 Varialbe 타입으로 리턴합니다. 
편향은 작은 양수(0.1)로 초기화하고 Varialbe 타입으로 리턴합니다. 

In [5]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

2D 컨볼루션과 맥스풀링을 정의하는 함수를 구현합니다.

텐서플로에서는 컨볼루션 관련 함수를 제공합니다. 함수명 끝 2d는 차원을 뜻합니다. 
stride는 1로 하고 패딩은 0으로 하는 컨볼루션 레이어를 만드는 함수를 정의합니다.
strides 옵션 NHWC의 정의는 다음과 같습니다. 
N: number of images in the batch
H: height of the image
W: width of the image
C: number of channels of the image (ex: 3 for RGB, 1 for grayscale...)

padding은 경계 처리 방법을 정이하는 옵션으로 다음 두 가지가 있습니다.
valid: 유효한 영역만 출력이 됩니다. 따라서 출력 이미지 사이즈는 입력 사이즈보다 작습니다.
same: 출력 이미지 사이즈가 입력 이미지 사이즈와 동일합니다.


그리고 2x2 맥스 풀링 레이어를 위한 함수를 정의합니다.

In [6]:
def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

이 예제는 윈도 크기가 5x5인 32개의 필터를 사용합니다. 
따라서 우리는 구조가 [5,5,1,32]인 가중치 행렬 W를 저장할 텐서를 정의해야 합니다. 
처음 두 개의 차원은 윈도의 크기이며 세 번째는 컬러 채널로 우리 예제에서는 1입니다.
마지막 차원은 얼마나 많은 특징을 사용할 것인지를 정의하는 것입니다.
편향은 필터수인 32로 정의합니다.

In [7]:
W_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32])

Instructions for updating:
Colocations handled automatically by placer.


첫번째 컨볼루션 레이어를 만들기 위해 
학습 데이터(x_image)에 대해 합성곱을 적용하고 편향을 더해줍니다(목표함수 수행).
그리고 활성화 함수로 ReLU(Rectified Linear Unit)를 적용합니다.
다음으로 출력 값을 구하기 위해 맥스 풀링을 적용합니다.

In [8]:
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

활성화 함수를 수행한 다음 마지만 차원이 1에서 32로 변경 되었습니다. 특징맵의 수를 의미합니다.
SAME 패딩이므로 컨볼루션으로는 차원이 변경되지 않고 풀링 단계에서 스트라이드에 따라 차원이 반으로 줄어듭니다.

In [9]:
print(x_image.get_shape())
print(h_conv1.get_shape())
print(h_pool1.get_shape())

(?, 28, 28, 1)
(?, 28, 28, 32)
(?, 14, 14, 32)


두 번째 은닉층 컨볼루션 레이어와 풀링 레이어를 구현하겠습니다.
5 * 5 윈도에 64개의 필터를 갖는 두 번째 합성곱 계층을 만들겠습니다. 
이때는 이전 계층의 출력 값의 크기(32)를 채널의 수로 넘겨야 합니다.(특징맵 수)
SAME 패딩이므로 콘볼루션으로는 차원이 변경되지 않고 풀링 단계에서 스트라이드에 따라 차원이 반으로 줄어든다.
14x14 크기 행렬인 h_pool1에 스트라이드 1로 5x5 윈도를 적용하여 합성곱 계층을 만들었고, 맥스 풀링까지 거쳐 크기는 7x7이 됩니다.

In [13]:
W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

print(h_conv2.get_shape())
print(h_pool2.get_shape())

(?, 14, 14, 64)
(?, 7, 7, 64)


다음 단계는 소프트맥스 계층에 주입하기 위해 7x7 출력 값을 완전 연결 계층에 연결합니다. 
전체 이미지를 처리하기 위해서는 1024개의 뉴런을 사용하도록 하겠습니다. 
이 경우 가중치와 편향 텐서는 다음과 같습니다.

마지막 소프트맥스 레이어에 연결하기 위해 완전연결 레이어를 추가합니다. 
이전 콘볼루션의 레이어의 결과 텐서를 다시 1차원 텐서로 변환하여 렐루 활성화 함수에 전달합니다.

In [14]:
W_fc1 = weight_variable([7 * 7 * 64, 1024])
b_fc1 = bias_variable([1024])

print(W_fc1.get_shape())
print(b_fc1.get_shape())

(3136, 1024)
(1024,)


이 텐서의 첫 번째 차원은 두 번째 합성곱 계층의 7x7 크기의 64개 필터를 뜻하며, 
두 번째 차원은 우리가 임의로 선택한 뉴런의 개수(여기서는 1024)입니다.
이제 텐서를 벡터로 변환합니다. 
소프트맥스 함수는 이미지를 직렬화해서 벡터 형태로 입력해야 합니다. 
이를 위해 가중치 행렬 W_fc1과 일차원 벡터를 곱하고 편향 b_fc1을 더한 후 렐루 활성화 함수를 적용합니다.

In [15]:
h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

다음 단계는 드롭아웃(dropout)이라는 기법을 통해 신경망에서 필요한 매개변수 수를 줄이는 것입니다. 
이는 노드를 삭제하여 입력과 출력 사이의 연결을 제거하는 것입니다. 
어떤 뉴런을 제거하고 어떤 것을 유지할지는 무작위로 결정됩니다. 
뉴런이 제거되거나 그렇지 않을 확률을 코드로 처리하지 않고 텐서플로에 위임할 것입니다.
소프트맥스 계층 전에 tf.nn.dropout 함수를 사용하여 드롭아웃을 적용합니다. 
그 전에 뉴런이 드롭아웃되지 않을 확률을 저장할 플에이스홀더를 만듭니다.
드롭아웃 자세한 내용은 https://ynebula.tistory.com/36 참고바랍니다.

In [21]:
keep_prob = tf.placeholder("float")
h_fc1_drop = tf.nn.dropout(h_fc1, rate=1-keep_prob)

마지막으로 소프트맥스 레이어를 추가합니다.

마지막 소프트맥스 레이어에 연결하기 위해 완전연결 레이어를 추가합니다. 
이전 컨볼루션의 레이어의 결과 텐서를 다시 1차원 텐서로 변환하여 렐루 활성화 함수에 전달합니다.


In [22]:
W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])

y_conv=tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)

5.3 모델 훈련 및 평가

경사 하강법 최적화 알고리즘을 ADAM 최적화 알고리즘으로 바꿨습니다(텐서틀로 API 문서에 따르면 ADAM 최적화 알고리즘이 특정한 장점을 가지고 있다고 가이드 함). 
또 앞서 언급한 드롭아웃 계층의 확률을 조절하는 추가 매개변수 keep_prob도 feed_dict 인수를 통해 전달합니다. 
크로스엔트로피와 최적화알고리즘, 평가를 위한 연산을 정의합니다.

In [23]:
cross_entropy = -tf.reduce_sum(y_*tf.log(y_conv))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))

세션을 시작하고 변수를 초기화 합니다.

In [24]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

20,000번 반복을 수행합니다.

In [ ]:
for i in range(20000):
    batch = next(shuffle_batch(X_train, y_train, 100))
    if i % 1000 == 0:
        train_accuracy = sess.run(accuracy, feed_dict={
                x:batch[0], y_: batch[1], keep_prob: 1.0})
        print("step %d, training accuracy %g"%(i, train_accuracy))
    sess.run(train_step, feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})

step 0, training accuracy 0.14
step 1000, training accuracy 0.98
step 2000, training accuracy 0.98
step 3000, training accuracy 0.99
step 4000, training accuracy 0.99
step 5000, training accuracy 0.99
step 6000, training accuracy 1
step 7000, training accuracy 1
step 8000, training accuracy 1


최종 정확도를 출력합니다.

In [ ]:
print("test accuracy %g"% sess.run(
        accuracy, feed_dict={x: X_test, y_: y_test, keep_prob: 1.0}))

In [10]:
import tensorflow as tf

#x = [None, [1, 2, 3, 4, 5, 6, 7, 8, 9]]
#x = [None, 9]
#t = tf.reshape(x, [1,3,3,1])

x = tf.placeholder("float", shape=[None, 784])
x_image = tf.reshape(x, [1,28,28,1])

print(t.get_shape())

(1, 3, 3, 1)


In [15]:
import tensorflow as tf

x = tf.constant([[1., 2., 3.],
                 [4., 5., 6.]])

x = tf.reshape(x, [1, 2, 3, 1])  # give a shape accepted by tf.nn.max_pool

valid_pad = tf.nn.max_pool(x, [1, 2, 2, 1], [1, 2, 2, 1], padding='VALID')
same_pad = tf.nn.max_pool(x, [1, 2, 2, 1], [1, 2, 2, 1], padding='SAME')

print(valid_pad.get_shape())
print(same_pad.get_shape())

#valid_pad.get_shape() == [1, 1, 1, 1]  # valid_pad is [5.]
#same_pad.get_shape() == [1, 1, 2, 1]   # same_pad is  [5., 6.]



(1, 1, 1, 1)
(1, 1, 2, 1)
